In [ ]:
# 코랩에 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import warnings
warnings.filterwarnings(action = "ignore")

# 데이터 처리 모듈
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

# 멀티프로세싱
import multiprocessing as mp
from multiprocessing import Pool

# 척도 변환 모듈(표준화, 정규화, 로버스트 정규화, 원핫인코딩(범주형에서 더미변수로 변환) 등)
from sklearn.preprocessing import *

# 결측값 관측 모듈
import missingno as msno

# 시각화 모듈
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

# 배열, 행렬 연산 모듈
import numpy as np

# 데이터 분할 모듈(훈련용 / 검증용 / 시험용)
from sklearn.model_selection import train_test_split

# 통계적 가설검정 및 계량화 모듈
# https://youtu.be/FtWEZw3kUho
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from scipy.interpolate import UnivariateSpline

# 타입 어노테이션 모듈
from typing import *

# 시간 관련 모듈
from tqdm import tqdm
from time import strptime, sleep

# 웹크롤링 / 스크래핑 관련 모듈
import requests
import io
import zipfile
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET

# 결측값 대체 및 특징 추출
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA, SparsePCA

# 연관규칙 분석 실행 모듈
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

Mounted at /content/drive


In [ ]:
class AprioriRuleswithEvent():

  def __init__(self, file_path: str, SUBSET = None):

    self.INDEX_NEWS_DATA = pd.read_csv(file_path, index_col = False, na_values = "NaN")
      # [file_path] "/content/drive/MyDrive/AfterLearnerProject/DataArchive/merge_derivative_news_data.csv"
      # [INDEX_PRINCIPAL_COMPONENT_x] 시계열 전처리한 5개 주가지수선형결합 제1주성분
      # [INDEX_PRINCIPAL_COMPONENT_y] 시계열 전처리한 5개 주가지수선형결합 제1주성분의 단위변동률
    self.INDEX_NEWS_DROPNA = self.INDEX_NEWS_DATA.dropna(subset = ["TAG_LIST"])
      # msno.dendrogram()
    self.SUBSET = SUBSET

  def compareEventDatetimewithNewsImportanceScore(self):

    # improtance와 특이 지점을 비교하기 위해 importance값 상위 20개 추출(특이지점과 흡사)
    display( self.INDEX_NEWS_DATA.sort_values(
        by = "IMPORTANCE", ascending = False).head(20)[["IMPORTANCE", "YYYYMMDD", "TAG_LIST"]] )


  def removeBlank(self):
    import re
    self.INDEX_NEWS_DROPNA["TAG_LIST"] = self.INDEX_NEWS_DROPNA["TAG_LIST"].apply(
        lambda tag: re.sub(" ", "", tag))
      # "\s{0,}"
      # re.sub(" +", "" , string)
      # " ".join(string.split())

  def mutateNewColumn(self):

    self.INDEX_NEWS_DROPNA["CHANGE_CATEGORY"] = (
        self.INDEX_NEWS_DROPNA["STABLE_or_DRAMATIC"].astype(str) + "_" + self.INDEX_NEWS_DROPNA["SIGN_CHANGE"].astype(str))
    self.INDEX_NEWS_DROPNA["TAG_SPLIT"] = self.INDEX_NEWS_DROPNA.TAG_LIST.str.split("|")
    print( "\n", Counter(self.INDEX_NEWS_DROPNA["CHANGE_CATEGORY"]) , "\n", Counter(self.INDEX_NEWS_DROPNA["YYYYMMDD"]) )


  def makeSubDataset_loadData(self, date_column: str, date_time: str):

    INDEX_NEWS_DATETIME = self.INDEX_NEWS_DROPNA[ self.INDEX_NEWS_DROPNA[date_column] == date_time ]
    self.SUBSET = INDEX_NEWS_DATETIME[["CHANGE_CATEGORY", "SUBCATEGORY"]].values
      # [column] "ITEM_NAME", "SUBCATEGORY"
    return self.SUBSET, INDEX_NEWS_DATETIME
      # [date_column] YYYYMMDD


  # Apriori Analysis
  def find_frequent_TAGsets(self, transaction_subset: pd.DataFrame, transaction_tagset: pd.Series,
                            drop_subset_list: List, drop_tagset_list: List,
                            minimum_support_threshold: float, minimum_confidence_threshold: float):

      transaction_subset_encoder = TransactionEncoder()
      transaction_tag_encoder = TransactionEncoder()

      transaction_subset_array = transaction_subset_encoder.fit_transform(transaction_subset)
      transaction_tag_array = transaction_tag_encoder.fit_transform(transaction_tagset)

      transaction_subset_dataframe = pd.DataFrame(
          data = transaction_subset_array, columns = transaction_subset_encoder.columns_)
      transaction_tagset_dataframe = pd.DataFrame(
          data = transaction_tag_array, columns = transaction_tag_encoder.columns_)

      transaction_subset_dataframe.drop(drop_subset_list, axis = 1, inplace = True)
      transaction_tagset_dataframe.drop(drop_tagset_list, axis = 1, inplace = True)
      transaction_dataframe = pd.concat(
          [transaction_subset_dataframe, transaction_tagset_dataframe],
          axis = 1, ignore_index = False)
      # print("\n", transaction_tag_encoder.columns_, end = ", ")

    # 이 지지도 이하의 규칙들은 배제하는 threshold (임계점 or 한계치)를 설정
      frequent_TAG_sets = apriori(
          transaction_dataframe,
          min_support = minimum_support_threshold,
          use_colnames = True)

    # apriori에 최소 지지도를 적용한 빈발원소집합, frequent_TAG_sets에서
    # [metric][confidence] 품목 사이의 연관정 정도를 측정하기 위해 사용되는 지표
    # [metric][lift] 두 사건이 서로 독립일 때와 비교해 얼마나 발생했는가에 대해 비율
    # print(apriori.__doc__)
      apriori_rules = association_rules(
          frequent_TAG_sets, metric = "confidence", min_threshold = minimum_confidence_threshold)
      return apriori_rules


##### 9가지 시점 통합 데이터프레임을 활용한 연관규칙분석 결과표

In [ ]:
def main():
  differencingPCA = AprioriRuleswithEvent(
    file_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/marketindex_news_data.csv")
  differencingPCA.compareEventDatetimewithNewsImportanceScore()

In [ ]:
if __name__ == "__main__" :
    main()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,IMPORTANCE,YYYYMMDD,TAG_LIST
274882,191.81,2023-02-08,맥주| 로봇| 산업| 원전| 에너지| 반도체| 아파트| 건설| 전기| 드론| 배터리...
119371,173.46,2023-01-18,스판덱스| 중국| 코로나| 섬유| 산업| 요가복| 레깅스| 반도체| 지주사| 책임|...
210272,168.84,2023-02-01,게임| 리니지| 결제| 아이템| 산업| 디아블로| 캐릭터| 장비| 그래픽| PC| ...
209015,146.29,2023-02-01,2차전지| 글로벌| 전기차| 전극| 자동차| 장비| 에너지| 배터리팩| 주행거리| 가전
23007,145.48,2023-01-04,다이소| 일본| 유통| 편의점| CU| 손해| 로켓배송| 쇼핑| 검색
803259,143.49,2023-04-18,테마파크| 그림| 조선| 조명| 북한| 게임| 기억
342936,141.50,2023-02-16,엔터테인먼트| BTS| 산업| 에스파
71932,135.84,2023-01-11,전기| 전력| 채권| 정치| 민간| 에너지| LNG
564173,135.17,2023-03-16,바이오| 헬스케어| 창업| 치료제| 반도체| 백신| 임상| 글로벌| 환자| 휴미라|...
914824,135.11,2023-02-11,그림| 러시아| 동물| 향수| 캔버스| 부모| 문화


In [ ]:
financialApriori = AprioriRuleswithEvent(
file_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/merge_derivative_news_data.csv")

financialApriori.compareEventDatetimewithNewsImportanceScore()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,IMPORTANCE,YYYYMMDD,TAG_LIST
43463,168.84,2023-02-01,게임| 리니지| 결제| 아이템| 산업| 디아블로| 캐릭터| 장비| 그래픽| PC| ...
42206,146.29,2023-02-01,2차전지| 글로벌| 전기차| 전극| 자동차| 장비| 에너지| 배터리팩| 주행거리| 가전
4243,129.80,2023-01-25,코로나| 호텔| 중국| 면세점| 여행| 관광객| 화장품| 제주| 일본| 여권| 소비
60764,125.75,2023-02-02,우주| 공연| 축구| 스포츠| 경찰| 공기| 발사| 문화| 태풍| 조선| 겨울| 운동
19339,111.14,2023-01-26,가구| 전기| 가스
13673,107.20,2023-01-26,교육| 양육
14055,100.42,2023-01-26,일본| 캐릭터| 도깨비| 캔버스
7666,100.09,2023-01-25,기온| 추위| 바닷물| 난방| 겨울| 청주| 제주
5739,98.79,2023-01-25,제주| 공항| 기온
32676,96.63,2023-01-31,경찰| 테러


In [ ]:
financialApriori.removeBlank()
financialApriori.mutateNewColumn()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



 Counter({'DRAMATIC_Not Change': 57557, 'DRAMATIC_Sign Change': 18186}) 
 Counter({'2023-03-15': 10156, '2023-02-02': 9123, '2023-02-01': 9063, '2023-01-31': 8900, '2023-01-26': 8298, '2023-03-10': 8118, '2023-04-12': 7651, '2023-01-25': 7532, '2023-01-27': 6902})



$ zhang's \ metric = \frac{Support(A \& B) - Support(A) Support(B)}{ max[Support(AB) (1-Support(A)), Support(A)(Support(B)-Support(AB))]} $

```
# Define a function to compute Zhang's metric
def zhang(antecedent, consequent):
	# Compute the support of each book
	supportA = antecedent.____
	supportC = consequent.____

	# Compute the support of both books
	supportAC = np.____(antecedent, consequent).____

	# Complete the expressions for the numerator and denominator
	numerator = supportAC - supportA*supportC
	denominator = ___(supportAC*(1-supportA), supportA*(supportC-supportAC))

	# Return Zhang's metric
	return numerator / denominator
```

```
  def scanD(D, Ck, minSupport):
    
    ssCnt = {}
    for tid in D:
      for can in Ck:
        if can.issubset(tid):
          if not can in ssCnt:
            ssCnt[can] += 1
    
    retList = []
    supportData: Tuple = {}
    numItems = float( len(D) )

    for key in ssCnt:
      support = ssCnt[key] / numItems
      if support >= minSupport:
        retList.insert(0, key)
      supportData[key] = support

    return retList, supportData
```

## **2023-01-25** | DRAMATIC_Not Change
(AI)	(인공지능) | (인공지능)	(AI)

(은행)	(금리) | (대출)	(금리)


In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230125, merge_derivative_news_data_230125 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-25")
# pd.set_option("display.max_row", 10)
# Counter(merge_derivative_news_data_230125["SUBCATEGORY"])

apriori_results_230125: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    # [반복적으로 오류가 발생하는 지점]
    transaction_subset = merge_derivative_news_data_subset_230125,
      # **TypeError: AprioriRuleswithEvent.find_frequent_TAGsets() got multiple values for argument 'transaction_subset'**
      # ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value nan
      # TypeError: '<' not supported between instances of 'float' and 'str'
      # TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

    transaction_tagset = merge_derivative_news_data_230125["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Not Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = ["톰슨로이터"],

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230125.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230125[0].sort_values("confidence", ascending = False) )
display( apriori_results_230125[1].sort_values("confidence", ascending = False) )
display( apriori_results_230125[2].sort_values("confidence", ascending = False) )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(인공지능),(AI),0.011285,0.025491,0.009161,0.811765,31.844853,0.008873,5.177078,0.979653
5,(대출),(금리),0.017525,0.050451,0.007966,0.454545,9.009569,0.007082,1.740839,0.904865
6,(은행),(금리),0.023234,0.050451,0.008895,0.382857,7.588632,0.007723,1.538620,0.888876
0,(AI),(인공지능),0.025491,0.011285,0.009161,0.359375,31.844853,0.008873,1.543360,0.993934
7,(금리),(은행),0.050451,0.023234,0.008895,0.176316,7.588632,0.007723,1.185850,0.914354
4,(금리),(대출),0.050451,0.017525,0.007966,0.157895,9.009569,0.007082,1.166689,0.936242
3,(중국),(글로벌),0.058285,0.059480,0.008763,0.150342,2.527620,0.005296,1.106940,0.641777
2,(글로벌),(중국),0.059480,0.058285,0.008763,0.147321,2.527620,0.005296,1.104420,0.642592


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(인공지능),(AI),0.011285,0.025491,0.009161,0.811765,31.844853,0.008873,5.177078,0.979653
18,(배터리),(전기차),0.010754,0.017924,0.005178,0.481481,26.863100,0.004985,1.894005,0.973241
9,(대출),(금리),0.017525,0.050451,0.007966,0.454545,9.009569,0.007082,1.740839,0.904865
25,(치료제),(환자),0.013011,0.017658,0.005576,0.428571,24.270677,0.005346,1.719099,0.971438
10,(은행),(금리),0.023234,0.050451,0.008895,0.382857,7.588632,0.007723,1.538620,0.888876
3,(IT),(스마트폰),0.014737,0.015932,0.005576,0.378378,23.749550,0.005341,1.583066,0.972222
0,(AI),(인공지능),0.025491,0.011285,0.009161,0.359375,31.844853,0.008873,1.543360,0.993934
2,(스마트폰),(IT),0.015932,0.014737,0.005576,0.350000,23.749550,0.005341,1.515789,0.973402
17,(추위),(기온),0.016596,0.019118,0.005576,0.336000,17.574667,0.005259,1.477231,0.959016
24,(환자),(치료제),0.017658,0.013011,0.005576,0.315789,24.270677,0.005346,1.442522,0.976033


## **2023-01-26** | DRAMATIC_Not Change

(대출)	(금리) | (금리, 은행)	(대출)

(무인기)	(북한)	| (북한)	(무인기)

In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230126, merge_derivative_news_data_230126 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-26")

apriori_results_230126: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    transaction_subset = merge_derivative_news_data_subset_230126,

    transaction_tagset = merge_derivative_news_data_230126["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Not Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = ["톰슨로이터"],

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230126.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230126[0].sort_values("confidence", ascending = False) )
display( apriori_results_230126[1].sort_values("confidence", ascending = False) )
display( apriori_results_230126[2].sort_values("confidence", ascending = False) )

## **2023-01-27** | DRAMATIC_Not Change
(마스크)	(코로나19) | (코로나19) (마스크)

(전기차)	(중국)	| (전기차)	(글로벌)

(전기차)  (배터리)	| (배터리)	(전기차)

In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230127, merge_derivative_news_data_230127 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-27")

apriori_results_230127: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    # [반복적으로 오류가 발생하는 지점]
    transaction_subset = merge_derivative_news_data_subset_230127,

    transaction_tagset = merge_derivative_news_data_230127["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Not Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = ["톰슨로이터"],

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230127.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230127[0].sort_values("confidence", ascending = False) )
display( apriori_results_230127[1].sort_values("confidence", ascending = False) )
display( apriori_results_230127[2].sort_values("confidence", ascending = False) )

## **2023-01-31** | DRAMATIC_Not Change
(반도체)	(메모리) | (메모리)	(반도체) | (파운드리)	(반도체)

(반도체)	(글로벌) | (일본)	(중국) | (코로나19)	(중국) | (소비)	(중국)

(부동산)	(주택)	| (주택)	(부동산)

In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230131, merge_derivative_news_data_230131 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-01-31")

apriori_results_230131: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    transaction_subset = merge_derivative_news_data_subset_230131,

    transaction_tagset = merge_derivative_news_data_230131["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Not Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = ["톰슨로이터"],

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230131.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230131[0].sort_values("confidence", ascending = False) )
display( apriori_results_230131[1].sort_values("confidence", ascending = False) )
display( apriori_results_230131[2].sort_values("confidence", ascending = False) )

## **2023-02-01** | DRAMATIC_Sign Change

(중국)	(글로벌) | (리오프닝)	(중국)

(대출)	(금리)

(반도체)	(메모리)

In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230201, merge_derivative_news_data_230201 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-02-01")

apriori_results_230201: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    transaction_subset = merge_derivative_news_data_subset_230201,

    transaction_tagset = merge_derivative_news_data_230201["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Sign Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = ["톰슨로이터"],

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230201.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230201[0].sort_values("confidence", ascending = False) )
display( apriori_results_230201[1].sort_values("confidence", ascending = False) )
display( apriori_results_230201[2].sort_values("confidence", ascending = False) )

## **2023-02-02** | DRAMATIC_Sign Change

(스마트폰)	(갤럭시) | (갤럭시)	(스마트폰) | (카메라)	(갤럭시) | (디지털)	(플랫폼)

(금리)	(중앙은행) | (금융)  (금리) | (채권)	(금리)

(배터리)	(전기차)	|  (전기차)	(배터리)



(전기)	(가스)

In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230202, merge_derivative_news_data_230202 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-02-02")

apriori_results_230202: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    transaction_subset = merge_derivative_news_data_subset_230202,

    transaction_tagset = merge_derivative_news_data_230202["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Sign Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = ["톰슨로이터"],

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230202.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230202[0].sort_values("confidence", ascending = False) )
display( apriori_results_230202[1].sort_values("confidence", ascending = False) )
display( apriori_results_230202[2].sort_values("confidence", ascending = False) )

## **2023-03-10** | DRAMATIC_Not Change

(이머징마켓, 아시아)	(미국) | (미국, 이머징마켓)	(아시아) | (중국, 아시아)	(이머징마켓)

(EU, 미국) | (, 소재)	(미국)

(채권)	(금리)

In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230310, merge_derivative_news_data_230310 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-03-10")

apriori_results_230310: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    transaction_subset = merge_derivative_news_data_subset_230310,

    transaction_tagset = merge_derivative_news_data_230310["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Not Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = ["톰슨로이터"],

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230310.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230310[0].sort_values("confidence", ascending = False) )
display( apriori_results_230310[1].sort_values("confidence", ascending = False) )
display( apriori_results_230310[2].sort_values("confidence", ascending = False) )

## **2023-04-12** | DRAMATIC_Not Change
(미국, 아시아)	(이머징마켓) | (미국, 이머징마켓)	(아시아) | (아시아, 이머징마켓)	(미국)
(소재)	(미국) | (헬스케어)	(미국)	| (미세먼지)	(황사)
(중국, 이머징마켓)	(아시아) | (EU, 아시아)	(이머징마켓)

In [ ]:
# date_time 변수가 9번 바뀌기 때문에 클래스로 선언하여 반복작업의 일부 자동화
merge_derivative_news_data_subset_230412, merge_derivative_news_data_230412 = financialApriori.makeSubDataset_loadData(
    date_column = "YYYYMMDD", date_time = "2023-04-12")

apriori_results_230412: List = []

for threshold in [0.01, 0.0075, 0.005]:
  apriori_rules = financialApriori.find_frequent_TAGsets(
    transaction_subset = merge_derivative_news_data_subset_230412,

    transaction_tagset = merge_derivative_news_data_230412["TAG_SPLIT"],

    drop_subset_list = ["DRAMATIC_Not Change", "사회", "경제", "정치", "기술"],
    drop_tagset_list = [""],
      # drop_tagset_list = ["톰슨로이터"]
      # KeyError: "['톰슨로이터'] not found in axis"

    minimum_support_threshold = threshold,
    minimum_confidence_threshold = 0.05
  )
  apriori_results_230412.append(apriori_rules)

In [ ]:
pd.set_option("display.max_row", None)
display( apriori_results_230412[0].sort_values("confidence", ascending = False) )
display( apriori_results_230412[1].sort_values("confidence", ascending = False) )
display( apriori_results_230412[2].sort_values("confidence", ascending = False) )

___

## **CS_DATA**와 MARKET_INDEX_NEWS_DATA 8가지 시점별 병합

* SY's Q.
  
  시장 전체에 대한 AU뉴스와 종목과 연관된 AI뉴스를 구별하여 연관규칙 분석을 수행할 필요가 있지 않을까

* 2023.03.15일자는 어떤 일자이기에 AI뉴스의 TAG LIST(해시태그로 추정) 전체가 결측값인걸까

In [ ]:
class LoadData():

# 클래스(로컬공간) 내부에서 cs_data, apy_itm_hist_dat, market_index_data의 값
# 생성자를 활용하여 초기화하고 클래스 내부에 캡슐화
  def __init__(self,
               cs_data_path: str, apy_itm_hist_path: str):
    # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
    cs_chunk = pd.read_csv(cs_data_path, chunksize = 10**5, index_col = False, na_values = "NaN", encoding = "utf-8")
    self.cs_data = list(cs_chunk)
    self.apy_itm_hist_data = pd.read_csv(apy_itm_hist_path, index_col = False, na_values = "NaN", encoding = "utf-8")
      # [BAS_YM] (19.12 ~ 23.04) 기준연월
      # [APY_ITM] (19.12 ~ 23.04)간 공모주 청약 진행된 종목 번호
      # [APY_FIN_YM] 이전 3년(19.12 ~ 22.11)간 마지막으로 참여한 공모주 청약연월(X)
      # [APY_ITM_CNT] 이전 3년(19.12 ~ 22.11)간 공모주 청약에 참여한 종목 개수(X)


# parallelDataFrame()
# csv(쉼표로 분할된) 데이터 로드 시 병렬처리하는 메서드
# 실제로 수행할 정도로 데이터 크기가 크지는 않고 100,000행씩 chunking하면 해결됨
  def parallelDataFrame(self, data, function):
    # 코어 수 확인(multiprocessing as mp)
    mp.cpu_count()
    # 코어 수를 전역변수 선언
    global num_cores
    # 코어 수만큼 입력받은 데이터를 2차원 배열로 분할하여 멀티프로세싱(병렬처리)
    data_split = np.array_split(data, num_cores)
    # 코어 수만큼 pool을 생성
    pool = mp.Pool(num_cores)
    # 나누어진 데이터를 function으로 넘겨서 병렬처리 수행
    data = pd.concat(pool.map(
        function, iterable = data_split))
    pool.close()
    # 모두 완료될 때까지 대기
    pool.join
    return data


# loadData()
# 불러온 데이터를 반환(코드 구현 시 활용하고 나중에는 제거할 메서드)
  def loadData(self):
    return self.cs_data, self.apy_itm_hist_data

In [ ]:
DataInstance = LoadData(
    cs_data_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/cs_data.csv",
    apy_itm_hist_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/apy_itm_hist.csv",
)
  # [참고] 경로 너무 길어지니까 겹치는 경로는 os.path.join 활용하여 반복 줄일 수 있음

In [ ]:
consumer_data, apply_item_histogram, = DataInstance.loadData()
  # cs_data, apy_item_hist 데이터

In [ ]:
consumer_merge_data = pd.concat(consumer_data)
  # [참고] 텍스트 파일로 저장해도 1.23GB 용량은 줄지 않는바 수행하지 않음
  # cs_merge_data.to_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/cs_data.txt", index = False)

* 시계열 정보 : 4개월간의 자산 및 거래, 접속내역 데이터
* 후위 표시자로 시기 구분
* 자산내역 : 월말일 기준 데이터 제공

  M1(2022.12), M2(2023.01), M3(2023.02), M4(2023.03)
* 거래내역, 접속내역 : 월별 초순, 중순, 하순 동안의 데이터 집계내역 제공

  M2_1(2023.01 초순 1 ~ 10일), M2_2(2023.01 중순 11 ~ 20일), M2_3(2023.01 하순 21 ~ 말일)을 _1, _2, _3으로 구분


In [ ]:
pd.set_option("display.max_row", 20)

In [ ]:
_, column_length = consumer_merge_data.shape

# 총 552개 컬럼(dtypes: float64(325), int64(2), object(195))으로
# 컬럼별 타입 및 결측값 확인 반복문 수행 필요
for index in range(0, column_length, 50):
    consumer_merge_data.iloc[:, index : index + 50].info()

In [ ]:
msno.bar(consumer_merge_data)
  # [참고] 데이터 상세상 0과 결측(NULL)의 구분이 없다는 점에서
  # 함부로 결측값을 예측모델로 대체하면 안됨

___

#### 8가지 시점 통합 데이터프레임을 활용한 연관규칙분석 결과표

```
apriori_results[0].sort_values("support", ascending = False)
  # apriori_results[0][
      # ( apriori_results[0]["lift"] > 1 ) &
      # ( apriori_results[0]["confidence"] >= 0.4 ) &
      # ( apriori_results[0]["support"] >= 0.2 ) ]

apriori_results[1].sort_values("support", ascending = False)

apriori_results[2].sort_values("support", ascending = False)
```

___

#### 시행착오의 과정



```
  merge_derivative_news_data_drop["TAG_SPLIT"] = merge_derivative_news_data_drop["TAG_SPLIT"].apply(lambda x: str(x))
  merge_derivative_news_data_drop["TAG_SPLIT"] = merge_derivative_news_data_drop["TAG_SPLIT"].apply(eval)
  merge_derivative_news_data_tagset = list(itertools.chain*merge_derivative_news_data_drop["TAG_SPLIT"].values)
    # TypeError: can't multiply sequence by non-int of type 'type'

  merge_derivative_news_data_subsets = [
      [categorize, subcategory, item_name, tag_split[0] if isinstance(tag_split, list) else tag_split]
      for categorize, subcategory, item_name, tag_split in merge_derivative_news_data_subset
  1]
```

```
  # 범주화를 위해 만든 categorize 열을 4개로 범주화하기

  # merge_derivative_news_data_drop['categorize'] = pd.Categorical(merge_derivative_news_data_drop['categorize'], categories=['A', 'B', 'C', 'D'])
  # merge_derivative_news_data_drop['categorize'] = merge_derivative_news_data_drop['categorize'].cat.codes

  # float가 포함된 TAG_LIST의 형식을 문자열로 선언 후 원핫인코딩, categorize열 범주화

  # merge_derivative_news_data_drop['test'] = merge_derivative_news_data_drop['test'].astype(str)
  # category_groups = merge_derivative_news_data_drop.groupby('categorize')
  # transactions = [group['test'].tolist() for _, group in category_groups]

```

```
  # 병렬 처리

  # if __name__=="__main__":
      # num_threads=4
      # with Pool(processes=num_threads)as pool:
          # frequent_TAGsets=pool.map(find_frequent_TAGsets,transactions)

    MDAD_encoded = pd.DataFrame(transaction_array, columns=transaction_encoder.columns_)
    frequent_TAGsets=apriori(MDAD_encoded, min_support=0.1, use_colnames=True)
    return frequent_TAGsets

      
  if __name__=="__main__":
      num_threads=4
      with Pool(processes=num_threads)as pool:
          frequent_TAGsets=pool.map(find_frequent_TAGsets,transactions)

  # 결과출력
  # for i, frequent_TAGsets in enumerate(frequent_TAGsets):
      # print(f"frequent TAGsets for Category {i+1}:")
      # print(frequent_TAGsets)
      # print()

  # 보기 좋게 정리

  apiriory = pd.Dataframe(te_result, columns = te.columns_)
  print(tabulate(df.head(5), headers = 'keys', tablefmt = 'fancy_grid'))

  itemset = apriori(df, min_support = 0.1, use_colnames = True)
  itemset
  print(itemset, '\n')

```
